In [ ]:
# Install required packages if not already installed
# !pip install gradio openai soundfile
import gradio as gr
import openai
import os
import soundfile as sf

In [ ]:
# Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')  # or set directly as a string

In [ ]:
# System prompt for expertise
def get_system_prompt():
    return "You are an expert technical tutor. Answer questions clearly and concisely."

# Function to get answer from selected model (OpenAI or Ollama)
def get_answer(question, model, stream=False):
    if model == 'OpenAI':
        response = openai.ChatCompletion.create(
            model='gpt-4o',
            messages=[
                {'role': 'system', 'content': get_system_prompt()},
                {'role': 'user', 'content': question}
            ],
            stream=stream
        )
        if stream:
            return ''.join([chunk['choices'][0]['delta'].get('content', '') for chunk in response])
        else:
            return response['choices'][0]['message']['content']
    elif model == 'Ollama':
        # Placeholder for Ollama API call
        return 'Ollama response placeholder'
    else:
        return 'Model not supported.'

In [ ]:
# Speech-to-text using Gradio's audio input
def audio_to_text(audio):
    if audio is None:
        return ""
    data, samplerate = sf.read(audio)
    # Save as wav for OpenAI Whisper
    wav_path = "temp.wav"
    sf.write(wav_path, data, samplerate)
    with open(wav_path, "rb") as f:
        transcript = openai.Audio.transcribe("whisper-1", f)
    return transcript['text']

In [ ]:
# Gradio interface function with streaming and tool demo
def qa_interface(question, model, audio, stream, use_calculator):
    if audio is not None:
        question = audio_to_text(audio)
    if use_calculator and question:
        try:
            # Simple calculator tool: evaluate math expressions
            result = eval(question, {"__builtins__": {}})
            return f"Calculator result: {result}"
        except Exception:
            pass  # If not a math expression, fall back to LLM
    answer = get_answer(question, model, stream=stream)
    return answer

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Technical Q&A Assistant (Week 2)")
    with gr.Row():
        question = gr.Textbox(label="Technical Question", lines=2)
        audio = gr.Audio(source="microphone", type="filepath", label="Or Speak Your Question")
    model = gr.Radio(["OpenAI", "Ollama"], label="Model", value="OpenAI")
    stream = gr.Checkbox(label="Enable Streaming", value=False)
    use_calculator = gr.Checkbox(label="Use Calculator Tool (for math questions)", value=False)
    answer = gr.Textbox(label="Answer", lines=4)
    submit = gr.Button("Get Answer")
    submit.click(qa_interface, inputs=[question, model, audio, stream, use_calculator], outputs=answer)
demo.launch()